In [10]:
# Cell 1: 环境设置 & 路径切换
import sys
import os

project_root = "/home/wjw/lerobot"
sys.path.insert(0, project_root)
os.chdir(project_root)

print("当前工作目录:", os.getcwd())


当前工作目录: /home/wjw/lerobot


In [63]:
# Cell 2: 检查 /dev/ttyACM* 端口
print("当前可用的 /dev/ttyACM 端口:")
!ls /dev/ttyACM*

print("\n查看 /dev/serial/by-id/ 绑定:")
!ls -l /dev/serial/by-id/

input("请确认上面输出是否与 configs.py 中配置的端口匹配；若不匹配，请手动更新后再按 Enter 继续...")



当前可用的 /dev/ttyACM 端口:
/dev/ttyACM0  /dev/ttyACM1

查看 /dev/serial/by-id/ 绑定:
总计 0
lrwxrwxrwx 1 root root 13 Mar 19 19:44 usb-1a86_USB_Single_Serial_58FA095568-if00 -> ../../ttyACM1
lrwxrwxrwx 1 root root 13 Mar 19 19:44 usb-1a86_USB_Single_Serial_58FA096602-if00 -> ../../ttyACM0


请确认上面输出是否与 configs.py 中配置的端口匹配；若不匹配，请手动更新后再按 Enter 继续... 


''

In [12]:
# # Cell 3: 强制清除旧标定文件
# import shutil

# calib_path = "/home/wjw/lerobot/.cache/calibration/koch/"
# shutil.rmtree(calib_path, ignore_errors=True)
# print("已删除旧标定文件夹:", calib_path)


已删除旧标定文件夹: /home/wjw/lerobot/.cache/calibration/koch/


In [13]:
# Cell 4: 实例化 KochRobotConfig 并触发标定
from lerobot.common.robot_devices.robots.configs import KochRobotConfig
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot

# 不考虑摄像头，清空 cameras
robot_cfg = KochRobotConfig()
robot_cfg.cameras = {}

robot = ManipulatorRobot(robot_cfg)

print("开始连接机器人（若无标定文件，将自动进入标定流程）...")
robot.connect()
print("✅ 标定完成，标定文件保存至:", robot_cfg.calibration_dir)



开始连接机器人（若无标定文件，将自动进入标定流程）...
Connecting main follower arm.
Connecting main leader arm.
Missing calibration file '.cache/calibration/koch/main_follower.json'

Running calibration of koch main follower...

Move arm to zero position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_zero.webp


Press Enter to continue... 



Move arm to rotated target position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rotated.webp


Press Enter to continue... 



Move arm to rest position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rest.webp


...


Move arm to rotated target position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/leader_rotated.webp


Press Enter to continue... 



Move arm to rest position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/leader_rest.webp


Press Enter to continue... 



Calibration is done! Saving calibration file '.cache/calibration/koch/main_leader.json'
Activating torque on main follower arm.
✅ 标定完成，标定文件保存至: .cache/calibration/koch


In [2]:
# Cell 5: 查看标定结果
print("当前标定文件夹内容:")
!ls -l /home/wjw/lerobot/.cache/calibration/koch/


当前标定文件夹内容:
总计 8
-rw-rw-r-- 1 wjw wjw 374 Mar 19 19:19 main_follower.json
-rw-rw-r-- 1 wjw wjw 368 Mar 19 19:21 main_leader.json


In [1]:
# Cell 6: 检查标定数据文件是否生成
!ls -l ~/.cache/calibration/koch/


总计 0
-rw-rw-r-- 1 wjw wjw 0 Mar 13 19:16 test.json


In [59]:
robot.disconnect()
print("⚠️ 断开连接，等待 5 秒后重新连接...")
import time
time.sleep(5)
robot.connect()
print("✅ 机械臂已重新连接！")


⚠️ 断开连接，等待 5 秒后重新连接...
Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.
✅ 机械臂已重新连接！


In [3]:
robot.connect()
print("✅ 机械臂已重新连接！")


NameError: name 'robot' is not defined

In [4]:
# 读取主臂的状态
leader_status = robot.leader_arms["main"].read("Hardware_Error_Status")
leader_torque = robot.leader_arms["main"].read("Torque_Enable")
leader_position = robot.leader_arms["main"].read("Present_Position")

# 读取从臂的状态
follower_status = robot.follower_arms["main"].read("Hardware_Error_Status")
follower_torque = robot.follower_arms["main"].read("Torque_Enable")
follower_position = robot.follower_arms["main"].read("Present_Position")

# 打印结果
print("✅ 主臂硬件错误状态:", leader_status)
print("✅ 主臂扭矩状态:", leader_torque)
print("✅ 主臂当前关节位置:", leader_position)
print("✅ 从臂硬件错误状态:", follower_status)
print("✅ 从臂扭矩状态:", follower_torque)
print("✅ 从臂当前关节位置:", follower_position)

from lerobot.common.robot_devices.motors.dynamixel import TorqueMode

# 读取主从臂的当前扭矩状态
# leader_torque = robot.leader_arms["main"].read("Torque_Enable")
# follower_torque = robot.follower_arms["main"].read("Torque_Enable")

# print("🔍 主臂当前扭矩状态:", leader_torque)
# print("🔍 从臂当前扭矩状态:", follower_torque)

# # 仅在扭矩未开启时才执行启用操作
# if not all(t == 1 for t in leader_torque):
#     print("⚠️ 主臂扭矩未开启，正在启用...")
#     robot.leader_arms["main"].write("Torque_Enable", TorqueMode.ENABLED.value)
# else:
#     print("✅ 主臂扭矩已开启，无需重复操作。")

# if not all(t == 1 for t in follower_torque):
#     print("⚠️ 从臂扭矩未开启，正在启用...")
#     robot.follower_arms["main"].write("Torque_Enable", TorqueMode.ENABLED.value)
# else:
#     print("✅ 从臂扭矩已开启，无需重复操作。")

# 重新检查扭矩状态，确保成功
# leader_torque = robot.leader_arms["main"].read("Torque_Enable")
# follower_torque = robot.follower_arms["main"].read("Torque_Enable")

# assert all(t == 1 for t in leader_torque), "❌ 主臂扭矩未正确启用！"
# assert all(t == 1 for t in follower_torque), "❌ 从臂扭矩未正确启用！"

# print("✅ 主从臂扭矩已正确启用！")


NameError: name 'robot' is not defined

In [19]:
# Cell 6: 测试主从同步遥控
import time
import tqdm

print("开始主从同步测试，时长 30 秒，频率 200Hz...")
seconds = 30
frequency = 200
interval = 1 / frequency

for _ in tqdm.tqdm(range(seconds * frequency)):
    leader_pos = robot.leader_arms["main"].read("Present_Position")
    robot.follower_arms["main"].write("Goal_Position", leader_pos)
    time.sleep(interval)

print("✅ 主从臂同步测试完成！")



开始主从同步测试，时长 30 秒，频率 200Hz...


100%|███████████████████████████████████████| 6000/6000 [01:03<00:00, 94.60it/s]

✅ 主从臂同步测试完成！


In [20]:
from lerobot.common.robot_devices.motors.dynamixel import TorqueMode

# 关闭主臂的所有关节扭矩
robot.leader_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)

# 关闭从臂的所有关节扭矩
robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)

print("✅  主臂和从臂扭矩已关闭！")

leader_torque = robot.leader_arms["main"].read("Torque_Enable")
follower_torque = robot.follower_arms["main"].read("Torque_Enable")

print("🔍  主臂扭矩状态:", leader_torque)
print("🔍  从臂扭矩状态:", follower_torque)

assert all(t == 0 for t in leader_torque), "❌  主臂扭矩未正确关闭！"
assert all(t == 0 for t in follower_torque), "❌  从臂扭矩未正确关闭！"

print("✅  主从臂扭矩已确认关闭！")



✅  主臂和从臂扭矩已关闭！
🔍  主臂扭矩状态: [0 0 0 0 0 0]
🔍  从臂扭矩状态: [0 0 0 0 0 0]
✅  主从臂扭矩已确认关闭！


In [5]:

robot.disconnect()
print("✅  机械臂已断开连接")

NameError: name 'robot' is not defined

In [4]:
#robot.leader_arms["main"].write("Hardware_Error_Status", 0)
#robot.follower_arms["main"].write("Hardware_Error_Status", 0)
#print("✅ 硬件错误状态已重置！")


In [5]:
ls /dev/video*


/dev/video0  /dev/video1  /dev/video2  /dev/video3  /dev/video4  /dev/video5


In [5]:
!v4l2-ctl --list-devices


USB 2.0 Camera: USB Camera (usb-0000:00:14.0-1.4):
	/dev/video2
	/dev/video3
	/dev/media1

USB2.0 HD UVC WebCam: USB2.0 HD (usb-0000:00:14.0-7):
	/dev/video0
	/dev/video1
	/dev/media0



In [10]:
!ffplay /dev/video2




ffplay version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2003-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [11]:
import cv2
import matplotlib.pyplot as plt

camera_index = 2
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"❌ 无法打开摄像头 /dev/video{camera_index}")
else:
    print(f"✅ 成功打开摄像头 /dev/video{camera_index}")

# 读取一帧
ret, frame = cap.read()
cap.release()

if ret:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # OpenCV 读取的颜色是 BGR，需要转换为 RGB
    plt.imshow(frame)
    plt.axis("off")  # 不显示坐标轴
    plt.title("Camera Test")
    plt.show()
else:
    print("❌ 无法读取摄像头画面")





❌ 无法打开摄像头 /dev/video2
❌ 无法读取摄像头画面


[ WARN:0@516.475] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@516.573] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [6]:
from lerobot.common.robot_devices.robots.configs import KochRobotConfig
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot
from lerobot.common.robot_devices.cameras.configs import OpenCVCameraConfig

# ✅ 创建 Koch 机器人配置
robot_cfg = KochRobotConfig()

# ✅ 添加摄像头配置，假设摄像头设备是 /dev/video2
robot_cfg.cameras = {
    "wrist_camera": OpenCVCameraConfig(
        camera_index=2,  # 设备端口
        fps=30,          # 帧率
        width=640,       # 画面宽度
        height=480       # 画面高度
    )
}

# ✅ 实例化机器人对象，并连接
robot = ManipulatorRobot(robot_cfg)
robot.connect()

print("✅ 机器人连接成功，并已启用摄像头！")


Connecting main follower arm.
Connecting main leader arm.
Missing calibration file '.cache/calibration/koch/main_follower.json'

Running calibration of koch main follower...

Move arm to zero position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_zero.webp


Press Enter to continue... 



Move arm to rotated target position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rotated.webp


Press Enter to continue... 



Move arm to rest position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rest.webp


...


Move arm to rotated target position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/leader_rotated.webp


Press Enter to continue... 



Move arm to rest position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/leader_rest.webp


Press Enter to continue... 



Calibration is done! Saving calibration file '.cache/calibration/koch/main_leader.json'
Activating torque on main follower arm.
✅ 机器人连接成功，并已启用摄像头！


In [13]:
print("✅ 当前摄像头配置：", robot_cfg.cameras)


✅ 当前摄像头配置： {'wrist_camera': OpenCVCameraConfig(camera_index=2, fps=30, width=640, height=480, color_mode='rgb', channels=3, rotation=None, mock=False)}


In [2]:
import cv2
import matplotlib.pyplot as plt
import matplotlib

# # 设置字体（使用 SimHei 或其它支持中文的字体）
# matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
# matplotlib.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
# 获取摄像头对象
camera = robot.cameras["wrist_camera"]

# 读取一帧图像
frame = camera.read()

if frame is not None:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 转换颜色格式
    plt.imshow(frame)
    plt.axis("off")  # 隐藏坐标轴
    plt.title("image")
    plt.show()
else:
    print("❌ 无法读取摄像头画面")



NameError: name 'robot' is not defined

In [18]:
import cv2

camera = robot.cameras["wrist_camera"]

while True:
    frame = camera.read()
    if frame is not None:
        cv2.imshow("📷 机械臂摄像头画面", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # 按 'q' 退出
        break

cv2.destroyAllWindows()
#按q退出


In [25]:
import os

# 设定数据存储路径
data_dir = "/home/wjw/lerobot/data_collection"
os.makedirs(data_dir, exist_ok=True)

print(f"✅ 数据将保存到：{data_dir}")


✅ 数据将保存到：/home/wjw/lerobot/data_collection


In [26]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime

# 创建数据存储结构
data_log = []

# 读取摄像头
camera = robot.cameras["wrist_camera"]
cap = cv2.VideoCapture(camera.camera_index)

# 设置视频参数
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)

print("🎥 摄像头已准备好，开始录制数据...")



🎥 摄像头已准备好，开始录制数据...


[ WARN:0@1259.029] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@1259.127] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [ ]:
import time
from tqdm import tqdm

# 设定录制时长（秒） & 频率
record_seconds = 10
frequency = 30  # 30Hz 录制
interval = 1 / frequency

print(f"🚀 录制开始：{record_seconds}s, 频率 {frequency}Hz...")

for _ in tqdm(range(record_seconds * frequency)):
    # 读取主臂当前关节角度
    leader_position = robot.leader_arms["main"].read("Present_Position")
    
    # 读取从臂目标关节角度
    follower_position = robot.follower_arms["main"].read("Goal_Position")
    
    # 读取时间戳
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

    # 读取摄像头图像
    ret, frame = cap.read()
    if ret:
        # 保存图像文件
        image_path = os.path.join(data_dir, f"{timestamp}.jpg")
        cv2.imwrite(image_path, frame)
    else:
        image_path = "ERROR"

    # 记录数据
    data_log.append([timestamp, leader_position, follower_position, image_path])

    time.sleep(interval)

print("✅ 录制完成！")
cap.release()


In [ ]:
# 转换为 DataFrame
df = pd.DataFrame(data_log, columns=["timestamp", "leader_position", "follower_position", "image_path"])

# 保存 CSV 文件
csv_path = os.path.join(data_dir, "robot_data.csv")
df.to_csv(csv_path, index=False)

print(f"✅ 数据已保存至：{csv_path}")


In [ ]:
df = pd.read_csv(csv_path)
print(df.head())  # 查看前 5 行数据


In [ ]:
import matplotlib.pyplot as plt

img_path = df.iloc[0]["image_path"]
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # 转换颜色格式

plt.imshow(img)
plt.axis("off")
plt.title("Sample Image from Dataset")
plt.show()
